# Online Shopping DBMS - Omer Seyfeddin Koc

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Required Libraries
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext, StringVar
from PIL import Image, ImageTk
import requests
from io import BytesIO

def admin_login():
    # Retrieve username and password from the login interface
    username = username_entry.get()
    password = password_entry.get()

    # Hardcoded admin credentials
    admin_username = "admin"
    admin_password = "12345"

    # Check if entered credentials match the hardcoded admin credentials
    if username == admin_username and password == admin_password:
        messagebox.showinfo("Success", "Successful login. Welcome, Admin!")  # Show success message
        login_window.destroy()  # Close the login window
        main_program()  # Start the main program if login is successful
    else:
        messagebox.showerror("Error", "Invalid username or password")  

def main_program():
    global canvas  # Declare global variable 'canvas' for graph visualization
    canvas = None

    # Function to create a connection to the MySQL database
    def create_connection():
        try:
            # Attempt to connect to the database with specified parameters
            return pymysql.connect(host='localhost', user='root', password='', db='OnlineShopping')
        except pymysql.MySQLError as e:
            messagebox.showerror("Database Connection Error", e)
            return None

    # Function to execute a given SQL query
    def execute_query(conn, query):
        try:
            with conn.cursor() as cursor:
                cursor.execute(query)  # Execute the query
                conn.commit()  # Commit changes to the database
                messagebox.showinfo("Successful", "Query successfully executed.")  # Show success message
        except pymysql.MySQLError as e:
            messagebox.showerror("Query Error", e)  

    # Function to execute a SELECT query and return results as a pandas DataFrame
    def select_query_to_dataframe(conn, query):
        try:
            return pd.read_sql(query, conn)
        except pymysql.MySQLError as e:
            messagebox.showerror("SELECT Query Error", e)  
            return None

    # Function to run a stored MySQL procedure with optional parameters
    def run_procedure(proc_name, params=()):
        global last_df  # Reference the global variable to store query results
        try:
            conn = create_connection()  
            cursor = conn.cursor()
            cursor.callproc(proc_name, params)  
            data = cursor.fetchall()  
            last_df = pd.DataFrame(data, columns=[i[0] for i in cursor.description]) 
            output_text.delete('1.0', tk.END)  
            output_text.insert(tk.END, last_df) 
        except pymysql.MySQLError as e:
            messagebox.showerror("Procedure Error", e) 
        finally:
            if conn:
                conn.close()  # Close the database connection
                
    def get_table_data(table_name):
        # This function fetches data from any specified table in the database and displays it in the GUI.
        global last_df
        query = f"SELECT * FROM {table_name};"
        df = select_query_to_dataframe(create_connection(), query)
        last_df=df
        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, df)
        
    def get_addresses_by_state(state_name):
        # Fetches and displays all addresses from the database that match a given state.
        global last_df
        query = f"SELECT * FROM Address WHERE State = '{state_name}';"
        df = select_query_to_dataframe(create_connection(), query)
        last_df=df
        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, df)

    def plot_state_distribution():
        # Visualizes the distribution of addresses by state using a bar chart.
        global canvas
        query = "SELECT State, COUNT(*) as Count FROM Address GROUP BY State ORDER BY Count DESC LIMIT 15;"
        df = select_query_to_dataframe(create_connection(), query)
        fig, ax = plt.subplots(figsize=(12, 6))  # Grafik boyutunu daha da genişlet
        df.plot(kind='bar', x='State', y='Count', ax=ax)
        ax.set_xlabel('State')
        ax.set_ylabel('Number of Addresses')
        ax.set_title('Address Distribution by State')
        ax.tick_params(axis='x', rotation=90)  # X eksenindeki etiketleri 90 derece döndür
        plt.tight_layout()  # Grafikteki boşlukları optimize et
        if canvas:
            canvas.get_tk_widget().pack_forget()  # Remove the old canvas before creating a new one

        canvas = FigureCanvasTkAgg(fig, master=tab_address)
        canvas.draw()
        canvas.get_tk_widget().pack()

    def get_product_count_by_category():
        # Retrieves and displays the count of products in each category.
        global last_df
        run_procedure('GetProductCountByCategory')
        if last_df is not None:
            # Veriyi en çok üründen en aza göre sırala
            sorted_df = last_df.sort_values(by='ProductCount', ascending=False)
            last_df=sorted_df
            output_text.delete('1.0', tk.END)
            output_text.insert(tk.END, sorted_df)

    def get_products_by_category_name(category_name):
        # Fetches and displays products belonging to a specific category.
        run_procedure('GetProductsByCategory', [category_name])

    def get_order_details_by_email(email):
        # Retrieves and shows detailed order information for a given customer email.
        run_procedure('GetOrdersDetailByEmail', [email])

    def clear_output():
        # Clears the output display area in the GUI
        global canvas
        output_text.delete('1.0', tk.END)
        if canvas:
            canvas.get_tk_widget().pack_forget()  # This will remove the plot from the GUI
            canvas = None  # Reset the canvas variable

    def get_reviews_by_count(count, most_reviews=True):
        # Displays products with the most or least reviews based on the 'count' parameter.
        global last_df
        order = "DESC" if most_reviews else "ASC"
        query = f"""
        SELECT P.ProductID, P.Name, COUNT(PR.ProductReviewID) AS ReviewCount
        FROM Product P
        LEFT JOIN ProductReview PR ON P.ProductID = PR.ProductID
        GROUP BY P.ProductID, P.Name
        ORDER BY ReviewCount {order}
        LIMIT {count};
        """
        df = select_query_to_dataframe(create_connection(), query)
        last_df=df
        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, df)

    def get_worst_rated_sellers(num_sellers):
        # Retrieves and displays a specified number of sellers with the lowest average ratings.
        global last_df
        query = f"""
        SELECT S.SellerID, S.SellerName, COALESCE(AVG(PR.Rating), 0) AS AverageRating
        FROM Seller S
        LEFT JOIN Product P ON S.SellerID = P.SellerID
        LEFT JOIN ProductReview PR ON P.ProductID = PR.ProductID
        GROUP BY S.SellerID, S.SellerName
        ORDER BY AverageRating ASC
        LIMIT {num_sellers};
        """
        df = select_query_to_dataframe(create_connection(), query)
        last_df=df
        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, df)

    # Main Window
    root = tk.Tk()
    root.title("Online Shopping Database Management System")
    root.geometry("800x1000")

    # Program Title
    header_label = tk.Label(root, text="ONLINE SHOPPING DATABASE MANAGEMENT SYSTEM", font=("Helvetica", 24, "bold"))
    header_label.pack(pady=20)

    # Tabs
    tab_control = ttk.Notebook(root)

    # Welcome Tab
    tab_welcome = ttk.Frame(tab_control)
    tab_control.add(tab_welcome, text="Home")
    
    # Load Image From File
    try:
        image_url = "https://iili.io/JTNERae.png"
        response = requests.get(image_url)
        if response.status_code == 200:
            img_data = BytesIO(response.content)
            img = Image.open(img_data)
            img = img.resize((400, 400), Image.Resampling.LANCZOS)
            photo = ImageTk.PhotoImage(img)
            label = tk.Label(tab_welcome, image=photo)
            label.image = photo 
            label.pack(pady=10)
        else:
            print("Error: ", response.status_code)
    except Exception as e:
        print("Error occured: ", str(e))

    # Description Text and Font Settings
    description = """
    Welcome to the Online Shopping Database Management System.
    This program is a project for the DS5110 course at Northeastern University.
    Developed by Omer Seyfeddin Koc.
    """
    description_label = tk.Label(tab_welcome, text=description, justify=tk.CENTER, font=("Helvetica", 18))
    description_label.pack(pady=10)

    tab_control.pack(expand=1, fill="both")

    ####################################################################################################
    # Table Selection Tab
    tab_table_selection = ttk.Frame(tab_control)
    tab_control.add(tab_table_selection, text="Tables")

    tab_control.pack(expand=1, fill="both")

    # Contents of the Table Selection Tab
    tk.Label(tab_table_selection, text="Select a Table Name:").pack(pady=5)
    table_names = ["Address", "Customer", "Status", "Seller", "PaymentStatus",
                   "Product", "Category", "ProductCategory", "ProductReview",
                   "ShoppingCart", "CartProduct", "PaymentType", "Payment",
                   "Shipment", "Order", "OrderItem","OperationLog"]

    table_var = StringVar(root)
    table_var.set("Tables")
    tables_menu = tk.OptionMenu(tab_table_selection, table_var, *table_names)
    tables_menu.pack(pady=10)

    get_table_button = tk.Button(tab_table_selection, text="Retrieve Table",
                                 command=lambda: get_table_data(table_var.get()))
    get_table_button.pack(pady=5)

    separator_line_between_buttons = tk.Frame(tab_table_selection, height=2, bd=1, relief=tk.SUNKEN)
    separator_line_between_buttons.pack(fill=tk.X, padx=5, pady=5)

    button_description_label = tk.Label(tab_table_selection, text="You can use these buttons to clear the output.")
    button_description_label.pack(pady=5)

    clear_button = tk.Button(tab_table_selection, text="Clear Output", command=clear_output)
    clear_button.pack(pady=5)
    
    ####################################################################################################
    # Shipment Queries Tab
    tab_shipment = ttk.Frame(tab_control)
    tab_control.add(tab_shipment, text="Shipment")

    # Contents of the Shipment Tab
    tk.Label(tab_shipment, text="Enter the State Name:").pack(pady=5)
    state_entry_shipment = tk.Entry(tab_shipment)
    state_entry_shipment.pack(pady=5)

    get_state_shipments_button = tk.Button(tab_shipment, text="Get Total Shipments by State",
                                           command=lambda: run_procedure('TotalShipmentsByState', [state_entry_shipment.get()]))
    get_state_shipments_button.pack(pady=5)

    tk.Label(tab_shipment, text="Enter the Tracking Number:").pack(pady=5)
    tracking_entry = tk.Entry(tab_shipment)
    tracking_entry.pack(pady=5)

    get_tracking_info_button = tk.Button(tab_shipment, text="Get Customer Details by Tracking Number",
                                         command=lambda: run_procedure('GetCustomerDetailsByTrackingNumber', [tracking_entry.get()]))
    get_tracking_info_button.pack(pady=5)

    def get_shipping_data():
        global last_df
        selected_status = status_var.get()  # Kullanıcının seçtiği status'u alın
        query = f"SELECT * FROM {selected_status}Orders"
        df = select_query_to_dataframe(create_connection(), query)
        last_df = df
        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, df)

    # Status List
    status_var = tk.StringVar()
    status_var.set("Processing")  # Initial list is Processing

    status_label = tk.Label(tab_shipment, text="Select a Status:")
    status_label.pack(pady=5)

    status_options = ["Processing", "Shipped", "Out_for_Delivery", "Delivered", "Cancelled"]
    status_menu = tk.OptionMenu(tab_shipment, status_var, *status_options)
    status_menu.pack(pady=5)

    get_shipping_data_button = tk.Button(tab_shipment, text="Get Data by Status",
                                         command=lambda: get_shipping_data())
    get_shipping_data_button.pack(pady=5)

    separator_line_between_buttons = tk.Frame(tab_shipment, height=2, bd=1, relief=tk.SUNKEN)
    separator_line_between_buttons.pack(fill=tk.X, padx=5, pady=5)

    button_description_label = tk.Label(tab_shipment, text="You can use these buttons to clear the output")
    button_description_label.pack(pady=5)

    clear_button = tk.Button(tab_shipment, text="Clear Output", command=clear_output)
    clear_button.pack(pady=5)

    ####################################################################################################
    # Customer Tabs
    tab_customer = ttk.Frame(tab_control)
    tab_control.add(tab_customer, text="Customer")

    tab_control.pack(expand=1, fill="both")

    tk.Label(tab_customer, text="Enter the email address to retrieve customer details:").pack(pady=5)
    email_entry = tk.Entry(tab_customer)
    email_entry.pack(pady=5)

    tk.Label(tab_customer, text="Click below to view customer details:").pack(pady=5)
    get_customer_info_button = tk.Button(tab_customer, text="Get Customer Details",
                                         command=lambda: run_procedure('GetCustomerDetailsByEmail', [email_entry.get()]))
    get_customer_info_button.pack(pady=5)

    separator_line_between_buttons = tk.Frame(tab_customer, height=2, bd=1, relief=tk.SUNKEN)
    separator_line_between_buttons.pack(fill=tk.X, padx=5, pady=5)

    button_description_label = tk.Label(tab_customer, text="You can use these buttons to clear the output")
    button_description_label.pack(pady=5)

    clear_button = tk.Button(tab_customer, text="Clear Output", command=clear_output)
    clear_button.pack(pady=5)

    ####################################################################################################
    # Address Tabs
    tab_address = ttk.Frame(tab_control)
    tab_control.add(tab_address, text="Address")

    tab_control.pack(expand=1, fill="both")

    tk.Label(tab_address, text="Enter a State to retrieve addresses:").pack(pady=5)
    state_entry = tk.Entry(tab_address)
    state_entry.pack(pady=5)

    get_addresses_button = tk.Button(tab_address, text="Get Addresses",
                                     command=lambda: get_addresses_by_state(state_entry.get()))
    get_addresses_button.pack(pady=5)

    tk.Label(tab_address, text="Click to plot the distribution of addresses across states (Limit 15):").pack(pady=5)
    plot_distribution_button = tk.Button(tab_address, text="Plot State Distribution",
                                         command=plot_state_distribution)
    plot_distribution_button.pack(pady=5)

    separator_line_between_buttons = tk.Frame(tab_address, height=2, bd=1, relief=tk.SUNKEN)
    separator_line_between_buttons.pack(fill=tk.X, padx=5, pady=5)

    button_description_label = tk.Label(tab_address, text="You can use these buttons to clear the output.")
    button_description_label.pack(pady=5)

    clear_button = tk.Button(tab_address, text="Clear Output", command=clear_output)
    clear_button.pack(pady=5)

    ####################################################################################################
    # Category Tabs
    tab_category = ttk.Frame(tab_control)
    tab_control.add(tab_category, text="Category")

    tk.Label(tab_category, text="Click to retrieve the number of items per category:").pack(pady=5)
    get_product_count_button = tk.Button(tab_category, text="Get Product Count by Category",
                                         command=get_product_count_by_category)
    get_product_count_button.pack(pady=5)

    tk.Label(tab_category, text="Enter a category name to retrieve products:").pack(pady=5)
    category_name_entry = tk.Entry(tab_category)
    category_name_entry.pack(pady=5)

    get_products_button = tk.Button(tab_category, text="Get Products",
                                    command=lambda: get_products_by_category_name(category_name_entry.get()))
    get_products_button.pack(pady=5)

    def get_category_data():
        global last_df
        selected_category_name = tab_category_entry.get()
        query = f"SELECT GetCategoryCountForProduct ('{selected_category_name}')"
        df = select_query_to_dataframe(create_connection(), query)
        last_df = df
        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, df)

    # Tab Category Entry box
    tk.Label(tab_category, text="Enter a product name:").pack(pady=5)
    tab_category_entry = tk.Entry(tab_category)
    tab_category_entry.pack(pady=5)

    tab_category_button = tk.Button(tab_category, text="Get Total Category Count",
                                          command=lambda: get_category_data())
    tab_category_button.pack(pady=5)

    separator_line_between_buttons = tk.Frame(tab_category, height=2, bd=1, relief=tk.SUNKEN)
    separator_line_between_buttons.pack(fill=tk.X, padx=5, pady=5)

    button_description_label = tk.Label(tab_category, text="You can use these buttons to clear the output")
    button_description_label.pack(pady=5)

    clear_button = tk.Button(tab_category, text="Clear Output", command=clear_output)
    clear_button.pack(pady=5)

    ####################################################################################################
    # Order Tab
    tab_order = ttk.Frame(tab_control)
    tab_control.add(tab_order, text="Order")

    # Get email as input
    tk.Label(tab_order, text="Retrieve detailed order information for a specific customer using their email address.").pack(pady=5)
    customer_email_order_entry = tk.Entry(tab_order)
    customer_email_order_entry.pack(pady=5)

    get_order_details_button = tk.Button(tab_order, text="Get Order Details by Email",
                                         command=lambda: get_order_details_by_email(customer_email_order_entry.get()))
    get_order_details_button.pack(pady=5)

    # Case-1
    tk.Label(tab_order, text="Group and count all orders by payment type.").pack(pady=5)
    get_orders_by_payment_type_button = tk.Button(tab_order, text="Get Orders by Payment Type",
                                                  command=lambda: run_procedure('GetOrderCountByPaymentType'))
    get_orders_by_payment_type_button.pack(pady=5)

    # Case-2
    tk.Label(tab_order, text="Group and count all orders by their status.").pack(pady=5)
    get_orders_by_status_button = tk.Button(tab_order, text="Get Orders by Status",
                                            command=lambda: run_procedure('GetOrderCountByStatus'))
    get_orders_by_status_button.pack(pady=5)

    # Case-3
    tk.Label(tab_order, text="Group and count all orders by payment status.").pack(pady=5)
    get_orders_by_payment_status_button = tk.Button(tab_order, text="Get Orders by Payment Status",
                                                    command=lambda: run_procedure('GetOrderCountByPaymentStatus'))
    get_orders_by_payment_status_button.pack(pady=5)

    separator_line_between_buttons = tk.Frame(tab_order, height=2, bd=1, relief=tk.SUNKEN)
    separator_line_between_buttons.pack(fill=tk.X, padx=5, pady=5)

    button_description_label = tk.Label(tab_order, text="You can use these buttons to clear the output.")
    button_description_label.pack(pady=5)

    clear_button = tk.Button(tab_order, text="Clear Output", command=clear_output)
    clear_button.pack(pady=5)

    ####################################################################################################
    # Product Tabs
    tab_product = ttk.Frame(tab_control)
    tab_control.add(tab_product, text="Product")

    # Case-1
    tk.Label(tab_product, text="Enter Seller Name to Retrieve Their Products:").pack(pady=5)
    seller_name_entry = tk.Entry(tab_product)
    seller_name_entry.pack(pady=5)

    get_products_by_seller_button = tk.Button(tab_product, text="Get Products by Seller",
                                              command=lambda: run_procedure('GetProductsBySeller', [seller_name_entry.get()]))
    get_products_by_seller_button.pack(pady=5)

    # Case-2
    tk.Label(tab_product, text="Get the number of products for each seller.").pack(pady=5)
    get_product_count_by_seller_button = tk.Button(tab_product, text="Get Product Count by Seller",
                                                   command=lambda: run_procedure('GetProductCountBySeller'))
    get_product_count_by_seller_button.pack(pady=5)

    # Case-3
    tk.Label(tab_product, text="Retrieve all products.").pack(pady=5)
    get_all_products_button = tk.Button(tab_product, text="Get All Products",
                                        command=lambda: run_procedure('GetAllProducts'))
    get_all_products_button.pack(pady=5)

    # Function to apply a discount to all products
    def apply_discount():
        try:
            # Get the discount percentage from the user
            discount_percentage = float(discount_entry.get())
            connection = create_connection()

            if connection is not None:
                # Create a cursor for the connection
                cursor = connection.cursor()
                # Call the stored procedure to apply the discount to all products
                cursor.callproc('setDiscountOnAllProducts', [discount_percentage])
                connection.commit()
                messagebox.showinfo("Success", f"Discount of {discount_percentage}% applied successfully to all products.")
                cursor.close()
                connection.close()
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    # Add a section to apply a discount to all products
    tk.Label(tab_product, text="Apply Discount to All Products. Enter discount percentage:").pack()
    discount_entry = tk.Entry(tab_product)
    discount_entry.pack(pady=5)
    apply_discount_button = tk.Button(tab_product, text="Apply Discount",
                                      command=apply_discount)
    apply_discount_button.pack(pady=5)

    def reset_discounts():
        try:
            connection = create_connection()

            if connection is not None:
                cursor = connection.cursor()

                # Call the user-defined function to reset discounts on all products
                cursor.execute("SELECT resetDiscountsOnAllProducts()")
                updated_count = cursor.fetchone()[0]
                messagebox.showinfo("Success", f"{updated_count} products have had their discounts reset.")

                connection.commit()
                cursor.close()
                connection.close()
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    # Add a section to apply a discount to all products
    tk.Label(tab_product, text="Reset Discounts on All Products.").pack()
    reset_discounts_button = tk.Button(tab_product, text="Reset Discount",
                                      command=reset_discounts)
    reset_discounts_button.pack(pady=5)

    separator_line_between_buttons = tk.Frame(tab_product, height=2, bd=1, relief=tk.SUNKEN)
    separator_line_between_buttons.pack(fill=tk.X, padx=5, pady=5)

    button_description_label = tk.Label(tab_product, text="You can use these buttons to clear the output.")
    button_description_label.pack(pady=5)

    clear_button = tk.Button(tab_product, text="Clear Output", command=clear_output)
    clear_button.pack(pady=5)


    ####################################################################################################
    # Product Review Tab
    tab_product_review = ttk.Frame(tab_control)
    tab_control.add(tab_product_review, text="P. Review")

    # Case-1
    tk.Label(tab_product_review, text="Retrieve all products with their reviews.").pack(pady=5)
    get_all_products_with_reviews_button = tk.Button(tab_product_review, text="Get All Products with Reviews",
                                                     command=lambda: run_procedure('GetAllProductsWithReviews'))
    get_all_products_with_reviews_button.pack(pady=5)

    # Case-2
    tk.Label(tab_product_review, text="Enter a number to retrieve products with the most reviews:").pack(pady=5)
    top_review_entry = tk.Entry(tab_product_review)
    top_review_entry.pack(pady=5)

    get_top_reviews_button = tk.Button(tab_product_review, text="Get Products with Most Reviews",
                                       command=lambda: get_reviews_by_count(int(top_review_entry.get()), True))
    get_top_reviews_button.pack(pady=5)

    # Case-3
    tk.Label(tab_product_review, text="Enter a number to retrieve products with the least reviews:").pack(pady=5)
    least_review_entry = tk.Entry(tab_product_review)
    least_review_entry.pack(pady=5)

    get_least_reviews_button = tk.Button(tab_product_review, text="Get Products with Least Reviews",
                                         command=lambda: get_reviews_by_count(int(least_review_entry.get()), False))
    get_least_reviews_button.pack(pady=5)

    def get_rw_data():
        global last_df
        selected_product_name = tab_product_review_entry.get()
        query = f"SELECT GetReviewCountForProduct ('{selected_product_name}')"
        df = select_query_to_dataframe(create_connection(), query)
        last_df = df
        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, df)

    # Entry Box
    tk.Label(tab_product_review, text="Enter a product name:").pack(pady=5)
    tab_product_review_entry = tk.Entry(tab_product_review)
    tab_product_review_entry.pack(pady=5)
    tab_product_review_button = tk.Button(tab_product_review, text="Get Review Count",
                                          command=lambda: get_rw_data())
    tab_product_review_button.pack(pady=5)

    separator_line_between_buttons = tk.Frame(tab_product_review, height=2, bd=1, relief=tk.SUNKEN)
    separator_line_between_buttons.pack(fill=tk.X, padx=5, pady=5)

    button_description_label = tk.Label(tab_product_review, text="You can use these buttons to clear the output.")
    button_description_label.pack(pady=5)

    clear_button = tk.Button(tab_product_review, text="Clear Output", command=clear_output)
    clear_button.pack(pady=5)

    ####################################################################################################
    # Seller Tab
    tab_seller = ttk.Frame(tab_control)
    tab_control.add(tab_seller, text="Seller")

    # Case-1
    tk.Label(tab_seller, text="Retrieve all products with seller information.").pack(pady=5)
    get_all_products_with_sellers_button = tk.Button(tab_seller, text="Get All Products with Sellers",
                                                     command=lambda: run_procedure('GetAllProductsWithSellers'))
    get_all_products_with_sellers_button.pack(pady=5)

    # Case-2
    tk.Label(tab_seller, text="Get average rating for each product.").pack(pady=5)
    get_avg_rating_button = tk.Button(tab_seller, text="Get Average Ratings",
                                      command=lambda: run_procedure('GetAverageRatingPerProduct'))
    get_avg_rating_button.pack(pady=5)

    # Case-3
    tk.Label(tab_seller, text="Enter number of sellers to retrieve with the worst ratings.").pack(pady=5)
    num_sellers_entry = tk.Entry(tab_seller)
    num_sellers_entry.pack(pady=5)

    get_worst_sellers_button = tk.Button(tab_seller, text="Get Worst Rated Sellers",
                                         command=lambda: get_worst_rated_sellers(int(num_sellers_entry.get())))
    get_worst_sellers_button.pack(pady=5)

    separator_line_between_buttons = tk.Frame(tab_seller, height=2, bd=1, relief=tk.SUNKEN)
    separator_line_between_buttons.pack(fill=tk.X, padx=5, pady=5)

    button_description_label = tk.Label(tab_seller, text="You can use these buttons to clear the output.")
    button_description_label.pack(pady=5)

    clear_button = tk.Button(tab_seller, text="Clear Output", command=clear_output)
    clear_button.pack(pady=5)

    ####################################################################################################
    # SQL Tab
    tab_sql = ttk.Frame(tab_control)
    tab_control.add(tab_sql, text="SQL Query")
    tab_control.pack(expand=1, fill="both")

    # Use Text widget for SQL query input
    sql_query_text = tk.Text(tab_sql, wrap=tk.WORD, width=60, height=15)
    sql_query_text.pack(pady=10)

    # Create a button and connection here to execute the query
    def execute_sql_query():
        global last_df
        conn = create_connection()
        if conn:
            query = sql_query_text.get("1.0", tk.END)  # Retrieve the query from the text box
            try:
                with conn.cursor() as cursor:
                    cursor.execute(query)
                    conn.commit()
                    result = cursor.fetchall()  # Retrieve the query result
                    conn.close()

                    # Convert the query result to a DataFrame
                    column_names = [desc[0] for desc in cursor.description]
                    df = pd.DataFrame(result, columns=column_names)
                    last_df = df

                    # Add the result to a ScrolledText widget
                    output_text.delete(1.0, tk.END)  # Clear previous content
                    output_text.insert(tk.END, df.to_string())

                    messagebox.showinfo("Success", "Query executed successfully.")
            except pymysql.MySQLError as e:
                messagebox.showerror("Query Error", e)

    run_query_button = tk.Button(tab_sql, text="Run SQL Query", command=execute_sql_query)
    run_query_button.pack(pady=5)

    separator_line_between_buttons = tk.Frame(tab_sql, height=2, bd=1, relief=tk.SUNKEN)
    separator_line_between_buttons.pack(fill=tk.X, padx=5, pady=5)

    button_description_label = tk.Label(tab_sql, text="You can use these buttons to clear the output.")
    button_description_label.pack(pady=5)

    clear_button = tk.Button(tab_sql, text="Clear Output", command=clear_output)
    clear_button.pack(pady=5)


    ########################################################################
    def add_product():
        try:
            conn = create_connection()
            if conn is None:
                return
            cursor = conn.cursor()

            # Get the product information entered by the user
            product_name = product_name_entry.get()
            product_description = product_description_entry.get()
            product_price = float(product_price_entry.get())

            # Get the selected Seller from the combobox
            selected_seller = seller_combobox.get().split(':')[0]
            # Securely insert the product into the database
            insert_product_query = "INSERT INTO Product (Name, Description, Price, SellerID) VALUES (%s, %s, %s, %s)"
            cursor.execute(insert_product_query, (product_name, product_description, product_price, selected_seller))
            # Get the last inserted ProductID
            product_id = cursor.lastrowid  # For MySQL, use 'cursor.lastrowid'. For other databases, this line may vary.
            # Get the selected categories
            selected_categories = category_tree.selection()
            for category in selected_categories:
                category_id = category.split(':')[0]
                insert_product_category_query = "INSERT INTO ProductCategory (ProductID, CategoryID) VALUES (%s, %s)"
                cursor.execute(insert_product_category_query, (product_id, category_id))
            conn.commit()
            messagebox.showinfo("Success", "Product added successfully.")

            # Clear the input fields and selections
            product_name_entry.delete(0, tk.END)
            product_description_entry.delete(0, tk.END)
            product_price_entry.delete(0, tk.END)
            seller_combobox.set('')
            for category in selected_categories:
                category_tree.selection_remove(category)
            conn.close()
        except Exception as e:
            conn.rollback()
            messagebox.showerror("Error", f"An error occurred: {str(e)}")
            conn.close()

    def populate_sellers(seller_combobox):
        try:
            conn = create_connection()
            if conn is None:
                raise ValueError("Unable to connect to the database")
            cursor = conn.cursor()

            # Fetch the list of Sellers, assuming the column for seller names is 'Name' (replace with the actual column name)
            cursor.execute("SELECT SellerID, SellerName FROM Seller")  # Update 'Name' to the correct column name
            sellers = cursor.fetchall()
            # Populate the seller_combobox with retrieved values
            seller_combobox['values'] = [f"{seller[0]}: {seller[1]}" for seller in sellers]
            conn.close()
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while fetching sellers: {str(e)}")

    def populate_categories(category_tree):
        try:
            conn = create_connection()
            if conn is None:
                raise ValueError("Unable to connect to the database")
            cursor = conn.cursor()
            # Fetch the list of Categories from the Category table
            cursor.execute("SELECT CategoryID, CategoryName FROM Category")  # Replace 'Name' with the actual column name
            categories = cursor.fetchall()

            for category in categories:
                category_tree.insert('', 'end', iid=category[0], text=category[1])
            conn.close()
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while fetching categories: {str(e)}")

    def add_customer():
        try:
            conn = create_connection()
            if conn is None:
                return
            cursor = conn.cursor()

            # Get the address information entered by the user
            number = address_number_entry.get()
            street = address_street_entry.get()
            city = address_city_entry.get()
            state = address_state_entry.get()
            zipcode = address_zipcode_entry.get()

            # Securely insert the address into the database
            insert_address_query = "INSERT INTO Address (Number, Street, City, State, Zipcode) VALUES (%s, %s, %s, %s, %s)"
            cursor.execute(insert_address_query, (number, street, city, state, zipcode))
            address_id = cursor.lastrowid

            # Get the customer information entered by the user
            first_name = customer_first_name_entry.get()
            last_name = customer_last_name_entry.get()
            email = customer_email_entry.get()
            phone = customer_phone_entry.get()
            password = customer_password_entry.get()

            # Securely insert the customer into the database
            insert_customer_query = "INSERT INTO Customer (FirstName, LastName, Email, Phone, AddressID, Password) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(insert_customer_query, (first_name, last_name, email, phone, address_id, password))
            conn.commit()
            messagebox.showinfo("Success", "Customer added successfully.")

            # Clear the input fields
            customer_first_name_entry.delete(0, tk.END)
            customer_last_name_entry.delete(0, tk.END)
            customer_email_entry.delete(0, tk.END)
            customer_phone_entry.delete(0, tk.END)
            address_number_entry.delete(0, tk.END)
            address_street_entry.delete(0, tk.END)
            address_city_entry.delete(0, tk.END)
            address_state_entry.delete(0, tk.END)
            address_zipcode_entry.delete(0, tk.END)
            conn.close()
        except Exception as e:
            conn.rollback()
            messagebox.showerror("Error", f"An error occurred: {str(e)}")
            conn.close()


    # Initialize the Insert tab
    tab_insert = ttk.Frame(tab_control)
    tab_control.add(tab_insert, text="Insert")

    # Define product_form and customer_form, but do NOT pack them immediately
    product_form = ttk.Frame(tab_insert)
    customer_form = ttk.Frame(tab_insert)

    # Packing the frames side by side
    customer_form.pack(side='left', expand=True, fill='both', padx=10, pady=10)
    product_form.pack(side='right', expand=True, fill='both', padx=10, pady=10)

    # Create input fields for adding a product
    tk.Label(tab_insert, text="Product Name:").pack()
    product_name_entry = tk.Entry(tab_insert)
    product_name_entry.pack(pady=5)

    tk.Label(tab_insert, text="Product Description:").pack()
    product_description_entry = tk.Entry(tab_insert)
    product_description_entry.pack(pady=5)

    tk.Label(tab_insert, text="Product Price:").pack()
    product_price_entry = tk.Entry(tab_insert)
    product_price_entry.pack(pady=5)

    # Create a Combobox for selecting a Seller
    tk.Label(tab_insert, text="Select Seller:").pack()
    seller_combobox = ttk.Combobox(tab_insert)
    seller_combobox.pack(pady=5)

    # Create a Treeview for selecting Categories
    tk.Label(tab_insert, text="Select Categories:").pack()
    category_tree = ttk.Treeview(tab_insert, selectmode="extended")
    category_tree.pack()
    category_tree.heading("#0", text="Category")

    # Create a button to add a product
    add_product_button = tk.Button(tab_insert, text="Add Product", command=add_product)
    add_product_button.pack(pady=10)

    # Add the tab control to the window
    tab_control.pack(expand=1, fill="both")

    # Populate the Sellers and Categories
    populate_sellers(seller_combobox) # Pass seller_combobox as an argument
    populate_categories(category_tree) # Pass category_tree as an argument

    # Customer input fields
    tk.Label(customer_form, text="First Name:").pack()
    customer_first_name_entry = tk.Entry(customer_form)
    customer_first_name_entry.pack(pady=5)

    tk.Label(customer_form, text="Last Name:").pack()
    customer_last_name_entry = tk.Entry(customer_form)
    customer_last_name_entry.pack(pady=5)

    tk.Label(customer_form, text="Email:").pack()
    customer_email_entry = tk.Entry(customer_form)
    customer_email_entry.pack(pady=5)

    tk.Label(customer_form, text="Phone:").pack()
    customer_phone_entry = tk.Entry(customer_form)
    customer_phone_entry.pack(pady=5)

    # Add a password entry in the customer form
    tk.Label(customer_form, text="Password:").pack()
    customer_password_entry = tk.Entry(customer_form)
    customer_password_entry.pack(pady=5)

    # Address input fields
    tk.Label(customer_form, text="Address Number:").pack()
    address_number_entry = tk.Entry(customer_form)
    address_number_entry.pack(pady=5)

    tk.Label(customer_form, text="Street:").pack()
    address_street_entry = tk.Entry(customer_form)
    address_street_entry.pack(pady=5)

    tk.Label(customer_form, text="City:").pack()
    address_city_entry = tk.Entry(customer_form)
    address_city_entry.pack(pady=5)

    tk.Label(customer_form, text="State:").pack()
    address_state_entry = tk.Entry(customer_form)
    address_state_entry.pack(pady=5)

    tk.Label(customer_form, text="Zipcode:").pack()
    address_zipcode_entry = tk.Entry(customer_form)
    address_zipcode_entry.pack(pady=5)

    # Button to add customer
    add_customer_button = tk.Button(customer_form, text="Add Customer", command=add_customer)
    add_customer_button.pack(pady=10)

    # Add the tab control to the window
    tab_control.pack(expand=1, fill="both")

    ####################################################################################################
    def fetch_orders(order_listbox):
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("""
            SELECT o.OrderID, CONCAT(c.FirstName, ' ', c.LastName) AS CustomerName 
            FROM `Order` o 
            JOIN Customer c ON o.CustomerID = c.CustomerID;
        """)
        orders = cursor.fetchall()
        for order in orders:
            order_listbox.insert(tk.END, order)  # This will insert each order as a tuple in the Listbox
        conn.close()

    def populate_status_options(status_combobox):
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT StatusID, StatusName FROM Status")
        status_list = cursor.fetchall()
        # Create a list of status names to update the combobox values
        status_names = [status[1] for status in status_list]
        status_combobox['values'] = status_names
        conn.close()


    def update_order_status(order_listbox, status_combobox):
        selected_order = order_listbox.get(order_listbox.curselection())
        order_id = selected_order[0]  # Assuming the OrderID is the first element
        selected_status_name = status_combobox.get()

        # Now get the StatusID for the selected status name
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT StatusID FROM Status WHERE StatusName = %s", (selected_status_name,))
        status_id = cursor.fetchone()[0]

        # Update the order with the new status
        update_query = "UPDATE `Order` SET StatusID = %s WHERE OrderID = %s"
        cursor.execute(update_query, (status_id, order_id))
        conn.commit()
        messagebox.showinfo("Success", f"Order {order_id} status updated to {selected_status_name}.")
        conn.close()

    update_tab = ttk.Frame(tab_control)
    tab_control.add(update_tab, text="Update")

    # Instruction label for the order listbox
    order_listbox_label = tk.Label(update_tab, text="Select an order to update its status:")
    order_listbox_label.pack(side='top', fill='x')
    # Order listbox setup
    order_listbox = tk.Listbox(update_tab)
    order_listbox.pack(side='left', fill='both', expand=True)
    fetch_orders(order_listbox)  # Populate the listbox with orders

    # Instruction label for the status combobox
    status_combobox_label = tk.Label(update_tab, text="Choose a new status for the selected order:")
    status_combobox_label.pack(side='left', fill='x')
    # Status combobox setup
    status_combobox = ttk.Combobox(update_tab)
    status_combobox.pack(side='left', fill='x', expand=True)
    populate_status_options(status_combobox)  # Populate the combobox with status options

    # Update button setup
    update_button = tk.Button(update_tab, text="Update Status", command=lambda: update_order_status(order_listbox, status_combobox))
    update_button.pack(side='left')

    # Packing the tab control and starting the Tkinter event loop
    tab_control.pack(expand=1, fill="both")

    ####################################################################################################
    def fetch_customers(customer_listbox):
        # Clear the existing entries in the listbox
        customer_listbox.delete(0, tk.END)

        # Now fetch and display updated customers
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT CustomerID, CONCAT(FirstName, ' ', LastName), Email FROM Customer")
        customers = cursor.fetchall()
        for customer in customers:
            customer_listbox.insert(tk.END, f"ID: {customer[0]}, Name: {customer[1]}, Email: {customer[2]}")
        conn.close()

    def fetch_products(product_listbox):
        # Clear the existing entries in the listbox
        product_listbox.delete(0, tk.END)

        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT ProductID, Name FROM Product")
        products = cursor.fetchall()
        for product in products:
            product_listbox.insert(tk.END, f"ID: {product[0]}, Name: {product[1]}")
        conn.close()

    def delete_customer(customer_listbox):
        selected_customer = customer_listbox.get(customer_listbox.curselection())
        if not selected_customer:
            messagebox.showerror("Error", "No customer selected.")
            return

        customer_id = selected_customer.split("ID: ")[1].split(",")[0]  # Extract CustomerID

        conn = create_connection()
        cursor = conn.cursor()

        try:
            conn.begin()
            # Delete ShoppingCart entries linked to the customer
            cursor.execute("DELETE FROM ShoppingCart WHERE CustomerID = %s", (customer_id,))

            # Retrieve AddressID
            cursor.execute("SELECT AddressID FROM Customer WHERE CustomerID = %s", (customer_id,))
            address_id = cursor.fetchone()[0]

            # Delete the customer
            cursor.execute("DELETE FROM Customer WHERE CustomerID = %s", (customer_id,))

            # Check if the address is linked to other customers
            cursor.execute("SELECT COUNT(*) FROM Customer WHERE AddressID = %s", (address_id,))
            if cursor.fetchone()[0] == 0:
                # Delete the address if not linked to other customers
                cursor.execute("DELETE FROM Address WHERE AddressID = %s", (address_id,))

            # Commit transaction
            conn.commit()
            messagebox.showinfo("Success", f"Customer with ID {customer_id} and their address deleted.")

        except Exception as e:
            conn.rollback()
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

        finally:
            conn.close()
            # Refresh the customer listbox
            fetch_customers(customer_listbox)

    def delete_product(product_listbox):
        selected_product = product_listbox.get(product_listbox.curselection())
        if not selected_product:
            messagebox.showerror("Error", "No product selected.")
            return

        product_id = selected_product.split("ID: ")[1].split(",")[0]  # Extract ProductID
        conn = create_connection()
        cursor = conn.cursor()

        try:
            conn.begin()

            # Delete entries in ProductCategory linked to the product
            cursor.execute("DELETE FROM ProductCategory WHERE ProductID = %s", (product_id,))

            # Then, delete the product
            cursor.execute("DELETE FROM Product WHERE ProductID = %s", (product_id,))

            conn.commit()
            messagebox.showinfo("Success", f"Product with ID {product_id} deleted.")

        except Exception as e:
            conn.rollback()
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

        finally:
            conn.close()
            fetch_products(product_listbox)


    delete_tab = ttk.Frame(tab_control)
    tab_control.add(delete_tab, text="Delete")

    # Customer listbox and delete button
    customer_listbox = tk.Listbox(delete_tab)
    customer_listbox.pack(side='left', fill='both', expand=True)
    fetch_customers(customer_listbox)
    delete_customer_button = tk.Button(delete_tab, text="Delete Customer", command=lambda: delete_customer(customer_listbox))
    delete_customer_button.pack(side='left')

    # Product listbox and delete button
    product_listbox = tk.Listbox(delete_tab)
    product_listbox.pack(side='right', fill='both', expand=True)
    fetch_products(product_listbox)
    delete_product_button = tk.Button(delete_tab, text="Delete Product", command=lambda: delete_product(product_listbox))
    delete_product_button.pack(side='right')

    tab_control.pack(expand=1, fill="both")
    ####################################################################################################
    # Output Box
    output_text = scrolledtext.ScrolledText(root, height=14, width=100)
    output_text.pack(pady=5)

    # Global Last_DF
    last_df = None

    root.mainloop()

# Login Window
login_window = tk.Tk()
login_window.title("Login Screen")

# image Url
image_url = "https://iili.io/JTNERae.png"

try:
    response = requests.get(image_url)
    if response.status_code == 200:
        img_data = BytesIO(response.content)
        img = Image.open(img_data)
        img = img.resize((300, 300), Image.LANCZOS)
        photo = ImageTk.PhotoImage(img)
        logo_label = tk.Label(login_window, image=photo)
        logo_label.image = photo
        logo_label.pack(pady=10)
    else:
        print("Error: ", response.status_code)

except Exception as e:
    print("Error occured: ", str(e))

# Description label
description = "Welcome to the Admin Login"
description_label = tk.Label(login_window, text=description, font=("Helvetica", 12))
description_label.pack(pady=10)


# Login box and button
tk.Label(login_window, text="Username:").pack()
username_entry = tk.Entry(login_window)
username_entry.pack()

tk.Label(login_window, text="Password:").pack()
password_entry = tk.Entry(login_window, show="*")
password_entry.pack()

login_button = tk.Button(login_window, text="Login", command=admin_login)
login_button.pack()

login_window.mainloop()

/var/folders/sn/m91g82490jbfm36bn4bwq1m40000gn/T/ipykernel_2117/3245235719.py:1100: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((300, 300), Image.LANCZOS)
